In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("C:/Users/user/Downloads/news.csv")

In [2]:
df.head(5)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [7]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [8]:
df.duplicated().sum()

0

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
cls_dist=df['label'].value_counts()
cls_dist

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [13]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split

max_words = 1000  # Max vocabulary size
max_len = 10      # Max sequence length
embedding_dim = 50  # Dimension of the embedding layer
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.25, random_state=42)


In [14]:
# Tokenize the texts
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
# Pad the sequences to the same length
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post')
print("Padded Training Data:\n", X_train_padded)


Padded Training Data:
 [[ 68   1  81 ...   1   5 193]
 [ 34 906   8 ...   1 870   1]
 [ 77   2  54 ... 211   1   1]
 ...
 [129  35 200 ...   1  24   1]
 [  1   2   1 ...   1   1   1]
 [  1   5   1 ...   5   1   1]]


In [18]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(64, return_sequences=False))  # You can increase the number of units if needed
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None,max_len))


c:\Users\user\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [19]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 10, 50)         │        50,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,505 (310.57 KB)

 Trainable params: 79,505 (310.57 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)


In [25]:
# Train the model
epochs = 10
history = model.fit(X_train_padded,y_train_encoded, epochs=epochs, validation_data=(X_test_padded,y_test_encoded), verbose=2)


Epoch 1/10
149/149 - 2s - 11ms/step - accuracy: 0.8369 - loss: 0.3631 - val_accuracy: 0.7393 - val_loss: 0.5712
Epoch 2/10
149/149 - 1s - 10ms/step - accuracy: 0.8503 - loss: 0.3392 - val_accuracy: 0.7405 - val_loss: 0.5874
Epoch 3/10
149/149 - 1s - 10ms/step - accuracy: 0.8621 - loss: 0.3103 - val_accuracy: 0.7437 - val_loss: 0.6081
Epoch 4/10
149/149 - 1s - 10ms/step - accuracy: 0.8741 - loss: 0.2928 - val_accuracy: 0.7393 - val_loss: 0.6687
Epoch 5/10
149/149 - 1s - 10ms/step - accuracy: 0.8798 - loss: 0.2748 - val_accuracy: 0.7361 - val_loss: 0.6572
Epoch 6/10
149/149 - 1s - 10ms/step - accuracy: 0.8922 - loss: 0.2555 - val_accuracy: 0.7191 - val_loss: 0.8232
Epoch 7/10
149/149 - 1s - 10ms/step - accuracy: 0.8960 - loss: 0.2453 - val_accuracy: 0.7203 - val_loss: 0.8285
Epoch 8/10
149/149 - 1s - 10ms/step - accuracy: 0.9063 - loss: 0.2253 - val_accuracy: 0.7222 - val_loss: 0.8618
Epoch 9/10
149/149 - 1s - 9ms/step - accuracy: 0.9173 - loss: 0.2032 - val_accuracy: 0.7222 - val_loss: 

In [26]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded,y_test_encoded, verbose=2)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


50/50 - 0s - 6ms/step - accuracy: 0.7292 - loss: 0.9810
Test Accuracy: 72.92%
